In [12]:
import numpy as np
import pandas as pd
import pydicom
%matplotlib inline
import matplotlib.pyplot as plt
import keras 
from skimage.transform import resize

In [76]:
# This function reads in a .dcm file, checks the important fields for our device, and returns a numpy array
# of just the imaging data
def check_dicom(filename): 
    print('Load file {} ...'.format(filename))
    ds = pydicom.dcmread(filename)    
    print(f'Image of type {ds["Modality"].value} of body part {ds["BodyPartExamined"].value} from position {ds["PatientPosition"].value}')
    
    img = ds.pixel_array
    
    return img
    
    
# This function takes the numpy array output by check_dicom and 
# runs the appropriate pre-processing needed for our model input
def preprocess_image(img,img_size): 
    # rescale image
    proc_img = img * 1./255

    # add channels
    proc_img = np.expand_dims(proc_img, axis=-1)
    proc_img = np.repeat(proc_img, 3, axis=-1)

    # resize image
    proc_img = resize(proc_img, img_size)
    
    return proc_img

# This function loads in our trained model w/ weights and compiles it 
def load_model(model_path, weight_path):
    # load model
    with open(model_path, 'r') as json_file:
        model_json = json_file.read()
        
    model = keras.models.model_from_json(model_json)

    # load model's weights
    model.load_weights(weight_path)
    
    return model

# This function uses our device's threshold parameters to predict whether or not
# the image shows the presence of pneumonia using our trained model
def predict_image(model, img, thresh): 
    prediction = model.predict(np.expand_dims(img, axis=0))
    prediction = prediction[0][0] > thresh
    
    return prediction 

In [77]:
test_dicoms = ['test1.dcm','test2.dcm','test3.dcm','test4.dcm','test5.dcm','test6.dcm']

model_path = 'my_model.json'
weight_path = 'xray_class_my_model_1.best.weights.h5'

IMG_SIZE=(224,224) # This might be different if you did not use vgg16

my_model = load_model(model_path, weight_path)
thresh = 0.6293706

# use the .dcm files to test your prediction
for i in test_dicoms:
    img = np.array([])
    img = check_dicom(i)
    
    if img is None:
        continue
        
    img_proc = preprocess_image(img,IMG_SIZE)
    pred = predict_image(my_model,img_proc,thresh)
    print(pred)

Load file test1.dcm ...
Image of type DX of body part CHEST from position PA
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 124ms/step
False
Load file test2.dcm ...
Image of type DX of body part CHEST from position AP
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
False
Load file test3.dcm ...
Image of type DX of body part CHEST from position AP
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
False
Load file test4.dcm ...
Image of type DX of body part RIBCAGE from position PA
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
False
Load file test5.dcm ...
Image of type CT of body part CHEST from position PA
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
False
Load file test6.dcm ...
Image of type DX of body part CHEST from position XX
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
False
